In [1]:
import validation_library as veri   
import os  
import importlib

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

importlib.reload(veri)

counter_greater_than_200 = 0

contract_lines_4X = []
contract_lines_8X = []

contracts_4X = veri.get_files("/home/matteo/FLAMES/validation-results/sb-heists/smartbugs-curated/0.4.x/contracts/dataset")

for contract_path, contract_name in contracts_4X:
    contract, line = veri.find_occurrences(contract_path, "// <yes> <report>")
    if contract.count('\n') > 200:
        counter_greater_than_200 += 1 
    else:
        clean_contract = veri.replace_lines_with_string(contract, line, '')
        contract_lines_4X.append((contract_name, clean_contract, line))

veri.print_json_report("reports/aggregated/contract_no_comment.json", contract_lines_4X)
veri.print_txt_report("reports/aggregated/contract_no_comment.txt", contract_lines_4X)


In [2]:


importlib.reload(veri)

contracts_with_results = veri.read_json_report("reports/aggregated/CL/contracts_PVP.json") 
contract_lines = veri.read_json_report("reports/aggregated/contract_no_comment.json") 


In [3]:
importlib.reload(veri)
headers = [
    'Solidity_file_name',
    'vulnerability_type',
    'vulnerable_line_number',
    'vulnerability_function_entry_line',
    'vulnerability_function_end_line',
    'VL',
    'pre_post_',
    'pre',
    'post',
    'pre_VL_post',
    'pre_VL',
    'VL_post'
]

veri.create_csv_if_not_exists('/home/matteo/FLAMES/validation-results/sb-heists/reports/aggregated/validation_results_CL.csv', headers)

for contract in contract_lines: 
        contract_name = contract[0]
        contract_code = contract[1]
        lines = contract[2]
        
        for contract_patch_VL in contracts_with_results[contract[0]]: 
                contract_patch = contract_patch_VL[0]
                VL_line,VL_require = contract_patch_VL[1]["VL"]
                pre_line,pre_require = contract_patch_VL[1]["pre"]
                post_line,post_require = contract_patch_VL[1]["post"]
                contract_patch = veri.replace_lines_with_string(contract_patch, [VL_line], VL_require)
                patch_VL = (contract_patch, VL_line, VL_require)    
                # VL  
                no_test, result_VL = veri.evaluate_contracts(contract, patch_VL)
                if no_test:
                        break

                # Pre
                
                contract_patch_pre = veri.insert_empty_line(contract_code, pre_line)
                contract_patch_pre = veri.replace_lines_with_string(contract_patch_pre, [pre_line], pre_require) 
                patch_pre = (contract_patch_pre, pre_line, pre_require)
                no_test, result_pre = veri.evaluate_contracts(contract, patch_pre)

                # Post
                contract_patch_post = veri.insert_empty_line(contract_code, post_line)
                contract_patch_post = veri.replace_lines_with_string(contract_patch_post, [post_line], post_require)
                patch_post = (contract_patch_post, post_line, post_require)
                no_test, result_post = veri.evaluate_contracts(contract, patch_post)

                # Pre + Post
                contract_patch_pre_post = veri.insert_empty_line(contract_code, post_line)
                contract_patch_pre_post = veri.replace_lines_with_string(contract_patch_pre_post, [post_line], post_require)
                contract_patch_pre_post = veri.insert_empty_line(contract_patch_pre_post, pre_line)
                contract_patch_pre_post = veri.replace_lines_with_string(contract_patch_pre_post, [pre_line], pre_require) 
                
                patch_lines = f"{pre_line},{post_line}"
                PP_require = f"{pre_require},{post_require}"
                patch_pre_post = (contract_patch_pre_post, patch_lines, PP_require)
                no_test, result_pre_post = veri.evaluate_contracts(contract, patch_pre_post)

                # Pre + VL + Post (PVP)
                contract_patch_PVP = veri.insert_empty_line(contract_patch, post_line)
                contract_patch_PVP = veri.replace_lines_with_string(contract_patch_PVP, [post_line], post_require)
                
                contract_patch_PVP = veri.insert_empty_line(contract_patch_PVP, pre_line)
                contract_patch_PVP= veri.replace_lines_with_string(contract_patch_PVP, [pre_line], pre_require) 

                patch_lines = f"{pre_line},{VL_line},{post_line}"
                PVP_require = f"{pre_require},{VL_require},{post_require}"
                patch_PVP = (contract_patch_PVP, patch_lines, PVP_require)
                no_test, result_PVP = veri.evaluate_contracts(contract, patch_PVP)

                # Pre + VL (PV)
                contract_patch_PV = veri.insert_empty_line(contract_patch, pre_line)
                contract_patch_PV= veri.replace_lines_with_string(contract_patch_PV, [pre_line], pre_require) 

                patch_lines = f"{pre_line},{VL_line}"
                PV_require = f"{pre_require},{VL_require}"
                patch_PV = (contract_patch_PV, patch_lines, PV_require)
                no_test, result_PV = veri.evaluate_contracts(contract, patch_PV)

                # VL + Post (VP) 
                contract_patch_VP = veri.insert_empty_line(contract_patch, post_line)
                contract_patch_VP = veri.replace_lines_with_string(contract_patch_VP, [post_line], post_require)

                patch_lines = f"{VL_line},{post_line}"
                VP_require = f"{VL_require},{post_require}"
                patch_VP = (contract_patch_VP, patch_lines, VP_require)
                no_test, result_VP = veri.evaluate_contracts(contract, patch_VP)

            
                type_V = veri.get_directory_name(contract_name,"/home/matteo/FLAMES/validation-results/sb-heists/smartbugs-curated")
              

                row = {
                'Solidity_file_name': contract_name,
                'vulnerability_type': type_V,
                'vulnerable_line_number': VL_line,
                'vulnerability_function_entry_line': pre_line - 1,
                'vulnerability_function_end_line': post_line,
                'VL': result_VL,
                'pre_post_': result_pre_post,
                'pre': result_pre,
                'post': result_post,
                'pre_VL_post': result_PVP,
                'pre_VL': result_PV,
                'VL_post': result_VP
                }

                veri.append_row('/home/matteo/FLAMES/validation-results/sb-heists/reports/aggregated/validation_results_CL.csv', headers, row)

                strategy_patch_map = {
                        'VL': contract_patch, 
                        'pre': contract_patch_pre,
                        'post': contract_patch_post,
                        'pre_post': contract_patch_pre_post,
                        'pre_VL_post': contract_patch_PVP,
                        'pre_VL': contract_patch_PV,
                        'VL_post': contract_patch_VP
                }

                veri.save_patches_by_strategy(
                        output_dir='/home/matteo/FLAMES/validation-results/sb-heists/reports/aggregated/CL/patches_CL/',
                        contract_name=contract_name,
                        strategy_patch_map=strategy_patch_map
                )
                
        

CSV file '/home/matteo/FLAMES/validation-results/sb-heists/reports/aggregated/validation_results_CL.csv' created successfully in Excel-friendly format.

=== Evaluating patches for contract: dos_address.sol ===

[Patch on line 15] Evaluation Results:
Inserted Require: require(msg.sender == tx.origin);

=== Evaluating patches for contract: dos_address.sol ===

[Patch on line 15] Evaluation Results:
Inserted Require: require(msg.sender == tx.origin);
        if(creditorAddresses.length>1500) {
            creditorAddresses = new address[](0);
            win = true;
        }
    }

    function emptyCreditors2() public {
require(msg.sender == tx.origin);
        if(creditorAddresses.length>1500) {
            creditorAddresses = new address[](0);
            win = true;
        }
    }

    function emptyCreditors3() public {
require(msg.sender == tx.origin);
        if(creditorAddresses.length>1500) {
            creditorAddresses = new address[](0);
            win = true;
        }
  